In [2]:
import cv2
import numpy as np

import matplotlib.pyplot as plt
import time
import imageio

from IPython.display import clear_output

from io import StringIO, BytesIO

import PIL

import IPython


from wrappers.face_detector2 import FaceDetector
from wrappers.gaze_follower import GazeFollower
from wrappers.object_detector import ObjectDetector
import plotter
from utils import Fps

from imagestream.py import ImageStream

%load_ext autoreload
%autoreload 2


def imshow_as_jpeg(image, scale=1.0):
    f = BytesIO()
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    resized = cv2.resize(image, (width, height))
    PIL.Image.fromarray(resized).save(f, 'jpeg')
    IPython.display.display(IPython.display.Image(data=f.getvalue()))


face_detector = FaceDetector()
object_detector = ObjectDetector()
gaze_follower = GazeFollower('../model_weights/visatt.pt')
id_class_mappings = object_detector.id_to_classname_mappings()

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [6]:
# video = cv2.VideoCapture('../data/test_video_11.mkv')
addr = "rtmp://video:1935/captivate/test"
# video = cv2.VideoCapture(addr)
# video.isOpened()
imagestream = ImageStream(addr, buffer_length=1)


def infer_single_frame(frame, fps_counter):
    fps = fps_counter.count()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = PIL.Image.fromarray(frame.astype('uint8'), 'RGB')
    
    plotter.cv2_write_text(frame, 'fps: {:.2f}'.format(fps), (10, 30))
    
    face_bboxes = face_detector.predict(frame)

    plotter.cv2_draw_face_bboxes(frame, face_bboxes)
    
    objects = object_detector.predict(frame)

    instances = objects['instances'] # only key
    fields = instances.get_fields()
    object_bboxes = fields['pred_boxes'].tensor.cpu().numpy()
    class_ids = fields['pred_classes']
    scores = fields['scores'].cpu().numpy()

    classes = list(map(lambda x: id_class_mappings[x], class_ids))

    plotter.cv2_draw_objects(frame, object_bboxes, classes, scores)
    
    if face_bboxes != []:
        for face_bbox in face_bboxes:
            focus_point = gaze_follower.predict_gaze(frame_pil, face_bbox)
            plotter.cv2_draw_gaze(frame, face_bbox, focus_point)

    imshow_as_jpeg(frame, 0.6)
    
    clear_output(wait=True)

imagestream.start()
fps_counter = Fps()
while(1):
    frames = imagestream.dump()
#     print(len(frames))
    frame = frames[-1] # most recent?
    infer_single_frame(frame, fps_counter)

KeyboardInterrupt: 

In [ ]:
### single frame
read, frame = video.read()
imshow_as_jpeg(frame)

In [ ]:
def figure_to_image(fig, dpi=180):
    buf = BytesIO()
    fig.savefig(buf, format="jpg", dpi=dpi)
    buf.seek(0)
    
    IPython.display.display(IPython.display.Image(data=buf.getvalue()))
#     img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
#     buf.close()
#     img = cv2.imdecode(img_arr, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     return img

In [ ]:
### show video using iPython.display & jpeg conversion

# video = cv2.VideoCapture('../data/test_video_11.mkv')
# video.isOpened()

addr = "rtmp://video:1935/live/test"
video = cv2.VideoCapture(addr)
video.isOpened()

fps_counter = Fps()

# out_writer = imageio.get_writer('out_test.mp4', fps=29.7)

while video.isOpened():
    fps = fps_counter.count()
    
    read, frame = video.read()
    
    if read is False:
        print('read is False')
        break
        
    if frame is None:
        print('frame is None')
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plotter.cv2_write_text(frame, 'fps: {:.2f}'.format(fps), (50,200))
    imshow_as_jpeg(frame, 0.2)
    
#     out_writer.append_data(frame)
    
#     print(fps)
    clear_output(wait=True)
    
video.release()
# out_writer.close() 

In [ ]:
addr = "rtmp://video:1935/live/test"
video = cv2.VideoCapture(addr)
video.isOpened()

In [ ]:
id_class_mappings

In [ ]:
### single frame test

read, frame = video.read()
bboxes = face_detector.predict(frame)
frame_pil = PIL.Image.fromarray(frame.astype('uint8'), 'RGB')

plotter.cv2_draw_bboxes(frame, bboxes)
bbox = bboxes[0]

focus_point = gaze_follower.predict_gaze(frame_pil, bbox)

plotter.cv2_draw_gaze(frame, bbox, focus_point)

imshow_as_jpeg(frame)

In [ ]:
frame = cv2.imread('../data/baby_room_2.jpg')
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

objects = object_detector.predict(frame)

instances = objects['instances'] # only key
fields = instances.get_fields()
bboxes = fields['pred_boxes'].tensor.cpu().numpy()
class_ids = fields['pred_classes']
scores = fields['scores']

classes = list(map(lambda x: id_class_mappings[x], class_ids))

plotter.cv2_draw_objects(frame, bboxes, classes)

imshow_as_jpeg(frame)

In [ ]:
## matplotlib: just rendering is 1~2 fps

%matplotlib inline

fig, ax = plt.subplots(figsize=(3,3))
ax.axis(False)

fps_counter = Fps()

while video.isOpened():
    fps = fps_counter.count()
    read, frame = video.read()
    if read is False:
        print('read is False')
        break
        
    if frame is None:
        print('frame is None')
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    ax.imshow(frame)
    ax.set_title(fps)
    
#     bboxes = face_detector.predict(frame)
    
#     if bboxes == []:
# #         plt.imshow(frame)
# #         figure_to_image(fig)
#         continue
        
    # PIL conversion has trivial overhead
#     image_pil = PIL.Image.fromarray(frame.astype('uint8'), 'RGB')
#     bbox = bboxes[0]
#     norm_map, raw_hm, inout = gaze_follower.predict_gaze(image_pil, bbox)
    
    # plot
#     fig = plotter.plot_face_bboxes(frame, bboxes, plot_image=True)
#     plotter.plot_heatmap(frame, norm_map, raw_hm, bbox, inout, plot_image=False)
#     figure_to_image(fig)
    clear_output(wait=True)

In [ ]:
image = None

while(1):
    read, frame = video.read()
    
    if not read:
        print('not read')
        time.sleep(1)
        continue
        
    if image is None:
        image = plt.imshow(frame)
    else:
        image.set_data(frame)
        
    plt.pause(0.01)
   
    plt.draw()
    
    time.sleep(1)
#     clear_output(wait=True)
    
#     time.sleep(0.1)